In [1]:
import itertools

from algorithm import run_more_plots, accuracy

from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

from sklearn.cluster import SpectralClustering

In [11]:
X, y = datasets.load_iris(return_X_y=True)
X, y = shuffle(X, y, random_state=0)
n_clusters = len(set(y))

X = X - X.mean(0)
X = X / X.std(0)

In [12]:
sp = SpectralClustering(n_clusters=n_clusters, random_state=0)
y_sp = sp.fit_predict(X)
print('direct spectral nmi:', normalized_mutual_info_score(y, y_sp))

direct spectral nmi: 0.689514783373153


In [13]:
alphas = [0, 0.001, 0.01, 0.1, 1, 10, 50, 100]
lambda_lassos = [0, 0.001, 0.01, 0.1, 1, 10, 50, 100]

metrics = []
for alpha, lambda_lasso in itertools.product(alphas, lambda_lassos):
    out = run_more_plots(X, y, 20, alpha, lambda_lasso, 0.1, n_clusters=n_clusters, plot=False, is_print=True, auto=False)
    metrics.append(out)

In [14]:
for met in metrics:
    our_nmi = normalized_mutual_info_score(y, met['our_labels'])
    our_acc = accuracy(y, met['our_labels'])
    our_ari = adjusted_rand_score(y, met['our_labels'])
    
    sp_nmi = normalized_mutual_info_score(y, met['spectral_labels'])
    sp_acc = accuracy(y, met['spectral_labels'])
    sp_ari = adjusted_rand_score(y, met['spectral_labels'])
    
    print(f'our nmi: {our_nmi:.2f}, our ari: {our_ari:.2f}, sp nmi: {sp_nmi:.2f}, sp ari: {sp_ari:.2f}')

our nmi: 0.00, our ari: 0.00, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.18, our ari: 0.02, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.01, our ari: 0.00, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.00, our ari: 0.00, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.19, our ari: 0.03, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.18, our ari: 0.02, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.72, our ari: 0.56, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.73, our ari: 0.57, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.00, our ari: 0.00, sp nmi: 0.72, sp ari: 0.56
our nmi: 0.17, our ari: 0.02, s